## کتابخانه های مورد نیاز 

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep,time
import re
from numpy import nan as NaN

## توابع مورد نیاز 

In [4]:
#   زیاد بود تابع برایش تعریف کردمcss selectorرکوئست 
def try_css_selector(soup,css):
    
    try:
        return soup.select_one(css).text.strip()
        
    except AttributeError:
        return None
    


In [5]:
#ادغام دیکشنری
merge_dic=lambda x,y:x|y

In [6]:
# تبدیل اعشار ساعت به دقیقه
def convert_to_standard_time(x):
  try:
    int_=int(float(x))
    float_=float(x) - int_
    # print('int ',int_)
    # print('float ',float_)
    
    if float_ == 0.25 or float_ == 0.75:
         
        temp= str(int_) + ":"+ str(int( float_*60))
     
    else:
       temp = str(int_) + ":"+ str(int( float_*6)) + "0"
       
    return temp
  except:
       return None 
  


In [7]:
# برای اینکه بتوانم تایم هارا جدا کنم دقیقه هارا تبدیل به اعشار ساعت کردم
def time_(x):
  
  y=list()
  times=dict()
  try:
    
      
    if "-" in x:
      
      x=x.replace("-"," ")
      # print("- ",x)
    while True:
        if ":" in x:
          if ":30" in x:
            x=x.replace(":30",".5")
      
          elif ":45" in x:
            x=x.replace(":45",".75")
      
          elif ":15" in x:
            x=x.replace(":15",".25")
      
          elif ":00" in x:
             x=x.replace(":00",".0")
          # print(": ",x)
        elif ":" not in x: break
      
      
    
    for i in x.split():
      
      try:
         if float(i):
          y.append(i)
      except ValueError:
        pass
      # print("i",i)
      # print("y ",y)

    if len(y) == 2:
      times["time_open_weekday"]=convert_to_standard_time(y[0])
      times["time_close_weekday"]=convert_to_standard_time(y[1])
      times["time_open_weekend"]=None
      times["time_close_weekend"]=None

    elif len(y)==4:
      times["time_open_weekday"]=convert_to_standard_time(y[0])
      times["time_close_weekday"]=convert_to_standard_time(y[1])
      times["time_open_weekend"]=convert_to_standard_time(y[2])
      times["time_close_weekend"]=convert_to_standard_time(y[3])

  except:
      times["time_open_weekday"]=None
      times["time_close_weekday"]=None
      times["time_open_weekend"]=None
      times["time_close_weekend"]=None

  return times



In [2]:
# باگی در تایم ها بود لینک ها در خروجی ژوپیتر بود را در فایلی کپی کردم 

with open(r"link\link_cafe.txt",'r') as f:
    link_cafe = [line.rstrip() for line in f]


##دیزاین  سایت تغییر کرده است و خروجی ای به ما نمیدهد در 11/25 تست کردم

## رکوٍئست و استخراج اطلاعات (crawling)

In [ ]:
# نام شهر ها
html = requests.get('https://fidilio.com/coffeeshops/')
if html.status_code == 200:
    
    
    soup=BeautifulSoup(html.content,'html.parser')

    inputTag = soup.find_all("option")

    citys=list()
    for i in inputTag:
    
        city=i.get("value")
        if city.isalpha():
            citys.append(city)

# city="isfahan","shiraz","mashhad","tabriz","kish","tehran","ghom","arak","ahwaz","sabzevar","urmia","zanjan","qazvin","hamedan","karaj","kerman","bandarabbas"
citys=list(set(citys))
citys



In [ ]:
# لینک کل کافه ها همراه شهر آن
link_cafe=[]
for city in citys:
    break_=0
    
    for i in range(50):
        if break_>5:break
        html=requests.get("https://fidilio.com/coffeeshops/in/%s/?p=%d"%(city,i))
        if html.status_code == 200:
            soup=BeautifulSoup(html.content,'html.parser')
            print(True)
        

            for a in soup.find_all('a',class_="restaurant-link", href=True):
                sleep(0.2)
                if a["href"] in link_cafe:
                    break_ +=1
                    continue
                link_cafe.append((a["href"],city))
                # print("Found the URL:", a['href'])
                
                
        

In [ ]:
# درخاست به هر صفحه مختص به یک کافه و استخراج اطلاعات 

c=list()
counter=0
for href in range(len(link_cafe)):
    # sleep(5)
    html = requests.get("https://fidilio.com/%s"%link_cafe[href][0])
    if html.status_code ==  200:
    
    
        soup=BeautifulSoup(html.content,'html.parser')
        print(True)

        cafe=dict()
        cafe["name"]=try_css_selector(soup,"h1")
 
        cafe["city"]=link_cafe[href][1]
        cafe['address']=try_css_selector(soup,".fa-map-marker+ .note")
        try:
             cafe['phone']=int(try_css_selector(soup,".note a"))
        except (TypeError , ValueError):
            cafe['phone']=None
        try:
            cafe['price_ratio']=try_css_selector(soup,".price-class").count("$")
        except:
            cafe['price_ratio']=None
        
        temp_time=try_css_selector(soup,".note .note")
        
            
        
        cafe=merge_dic(cafe,time_(temp_time))  
        
            
        try:
           cafe["features"]=try_css_selector(soup,".venue-features-box").replace("\n",",")#.split("\n")
        except:
            cafe["features"]=None
        try:
            
            # استخراج امتیازات نظردهی به چهار مورد (کیفیت غذا ، دکوراسیون ، سرویس دهی و غیره)
            n_star=list()
            star=soup.find_all("div",attrs={"data-rateit-resetable":"false"})
            star=list(star)[:5]

            for i in star:
    
                temp1=re.search(r'.*"([\d\.\s]+)".*',str(i))
                if temp1:
                    n_star.append(float(temp1.group(1).strip()))
                
        
            cafe["score_food_quality"]=n_star[0]
            cafe["score_service"]=n_star[1]
            cafe["score_value_money"]=n_star[2]
            cafe["score_Decor_env"]=n_star[3]
            cafe["score_total"]=n_star[4]
        
        except:
            cafe["score_food_quality"]=None
            cafe["score_service"]=None
            cafe["score_value_money"]=None
            cafe["score_Decor_env"]=None
            cafe["score_total"]=None
            
        
        c.append(cafe)
        sleep(0.2)
        print(counter)
        counter+=1

In [22]:
# convert dict to DateFrame
df=pd.DataFrame()
nan=0
counter=0
for i in range(len(c)):  
        output=pd.DataFrame(c[i],index=[0])
        if output.isnull().sum().sum()>9:
            nan+=1
        else: 
            nan=0
            df=pd.concat([df,output],axis=0,ignore_index=True)
            
        print("nan ",nan)
        print("c ",counter)
        counter+=1
        

nan  0
c  0
nan  0
c  1
nan  0
c  2
nan  0
c  3
nan  0
c  4
nan  0
c  5
nan  0
c  6
nan  0
c  7
nan  0
c  8
nan  0
c  9
nan  0
c  10
nan  0
c  11
nan  0
c  12
nan  0
c  13
nan  0
c  14
nan  0
c  15
nan  0
c  16
nan  0
c  17
nan  0
c  18
nan  0
c  19


## datatime تبدیل فرمت استرینگ ساعت کاری به  

In [50]:
df['time_open_weekday']=df['time_open_weekday'].replace("24:00","00:00")
df['time_open_weekday']=df['time_open_weekday'].replace("24:30","00:30")
# df[df["time_open_weekday"]=="24:00"]
df.time_open_weekday = pd.to_datetime(df["time_open_weekday"]).dt.time

In [54]:
df['time_close_weekday']=df['time_close_weekday'].replace("24:00","00:00")
df['time_close_weekday']=df['time_close_weekday'].replace("24:30","00:30")
df.time_close_weekday = pd.to_datetime(df["time_close_weekday"]).dt.time

In [56]:
df['time_open_weekend']=df['time_open_weekend'].replace("24:00","00:00")
df['time_open_weekend']=df['time_open_weekend'].replace("24:30","03:00")
df.time_open_weekend = pd.to_datetime(df["time_open_weekend"]).dt.time

In [57]:
df['time_close_weekend']=df['time_close_weekend'].replace("24:00","00:00")
df['time_close_weekend']=df['time_close_weekend'].replace("24:30","00:30")
df.time_close_weekend = pd.to_datetime(df["time_close_weekend"]).dt.time

## ذخیره خروجی به فایل اکسل

In [58]:
df.to_excel("cafeha5.xlsx")